# Parcellate XCP-D denoised BOLD with a custom atlas

Users may want to parcellate their denoised BOLD data with an atlas that isn't built into XCP-D.
They _could_ simply use a NiftiLabelsMasker to parcellate the data, but this would ignore some of the extra steps that XCP-D applies, including the removal of missing voxels/vertices and the application of a minimum coverage threshold.

This notebook assumes that the atlas is in the same space and resolution as the denoised BOLD data.
If it is not, then the user will need to warp the atlas to the right space and resolution ahead of time.

In [ ]:
from xcp_d.interfaces.connectivity import CiftiConnect, NiftiConnect

In [ ]:
xcpd_dir = "/Users/taylor/Documents/datasets/ds003643/derivatives/xcp_d/sub-EN100/func"
fmriprep_dir = "/Users/taylor/Documents/datasets/ds003643/derivatives/fmriprep/sub-EN100/func"

custom_atlas = ""

filtered_file = os.path.join(xcpd_dir, "sub-EN100_task-lppEN_run-1_space-MNI152NLin6Asym_desc-denoised_bold.nii.gz")
mask = os.path.join(fmriprep_dir, "sub-EN100_task-lppEN_run-1_space-MNI152NLin6Asym_desc-brain_mask.nii.gz")
temporal_mask = os.path.join(xcpd_dir, "sub-EN100_task-lppEN_run-1_outliers.tsv")

min_coverage = 0.5  # use the same threshold you used for the XCP-D call
correlate = True

In [ ]:
nifti_atlas = ""

# Step 1. Mock up an atlas labels file
nifti_atlas_labels = ""

# Step 2. Warp the atlas to the same space as the BOLD file
transform_files = get_std2bold_xfms(nifti_file)

grab_first_volume = IndexImage(
    in_file=nifti_file,
    index=0,
)
gfv_results = grab_first_volume.run()

warp_atlases_to_bold_space = ApplyTransforms(
    interpolation="GenericLabel",
    input_image_type=3,
    dimension=3,
    reference_image=gfv_results.out_file,
    input_image=nifti_atlas,
    transforms=transform_files,
)
watbs_results = warp_atlases_to_bold_space.run()
warped_nifti_atlas = watbs_results.output_image

# Step 3. Parcellate the BOLD file
interface = NiftiConnect(
    filtered_file=nifti_file,
    mask=mask,
    temporal_mask=temporal_mask,
    atlas=warped_nifti_atlas,
    atlas_labels=nifti_atlas_labels,
    min_coverage=min_coverage,
    correlate=True,
)
results = interface.run()
timeseries_file = results.timeseries
correlations_file = resutls.correlations

In [ ]:
cifti_atlas = ""

# Step 1. Mock up an atlas labels file
cifti_atlas_labels = ""

# Step 2. Parcellate the CIFTI atlas
resample_atlas_to_data = CiftiCreateDenseFromTemplate(
    template_cifti=cifti_file,
    label=cifti_atlas,
)
ratd_results = resample_atlas_to_data.run()

parcellate_atlas = CiftiParcellate(
    direction="COLUMN",
    only_numeric=True,
    atlas_label=cifti_atlas,
    in_file=ratd_results.cifti_out,
    out_file="parcellated_atlas.pscalar.nii",
)
pa_results = parcellate_atlas.run()
parcellated_cifti_atlas = pa_results.out_file

cifti_atlas_int16 = cast_cifti_to_int16(ratd_results.cifti_out)

# Step 3. Parcellate the BOLD file
interface = CiftiConnect(
    data_file=cifti_file,
    temporal_mask=temporal_mask,
    atlas_file=cifti_atlas_int16,
    parcellated_atlas=parcellated_cifti_atlas,
    atlas_labels=cifti_atlas_labels,
    min_coverage=min_coverage,
    correlate=True,
)
results = interface.run()
timeseries_file = results.timeseries
timeseries_cifti_file = results.timeseries_ciftis
correlations_file = results.correlations
correlations_cifti_file = results.correlation_ciftis